In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(temperature=0.1) 

prompt = "Cost of $355.43 + $924.85 + $5000.9 + $1940.9 + $550.33 + $440.44 + $333.33 + $22222.22"

llm.invoke(prompt)

AIMessage(content='= $355.43 + $924.85 + $5000.9 + $1940.9 + $550.33 + $440.44 + $333.33 + $22222.22\n= $29,768.00', response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 51, 'total_tokens': 102}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8e0d45e4-6b2e-4016-a577-c59863208820-0')

In [7]:
def plus(a, b): 
    return a + b 

agent = initialize_agent(llm=llm, verbose=True, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, tools=[
    StructuredTool.from_function(
        func=plus, 
        name="Sum Calculator",
        description="Use this to perform sums of two numbers. this tool take two arguments, both should be numbers"
    )
])

agent.invoke(prompt)



> Entering new AgentExecutor chain...
Thought: I can use the Sum Calculator tool to find the total cost of all these amounts.
Action:
```
{
  "action": "Sum Calculator",
  "action_input": {"a": 355.43, "b": 924.85}
}
```
Observation: 1280.28
Thought:I have calculated the sum of the first two amounts correctly. Now, I will continue to calculate the total cost by adding the remaining amounts one by one using the Sum Calculator tool. Let's proceed with the next calculation. 
Action:
```
{
  "action": "Sum Calculator",
  "action_input": {"a": 1280.28, "b": 5000.9}
}
```
Observation: 6281.179999999999
Thought:I have calculated the sum of the first three amounts correctly. Now, I will continue to calculate the total cost by adding the remaining amounts one by one using the Sum Calculator tool. Let's proceed with the next calculation. 
Action:
```
{
  "action": "Sum Calculator",
  "action_input": {"a": 6281.18, "b": 1940.9}
}
```
Observation: 8222.08
Thought:I have calculated the sum of the

{'input': 'Cost of $355.43 + $924.85 + $5000.9 + $1940.9 + $550.33 + $440.44 + $333.33 + $22222.22',
 'output': 'The total cost of $355.43 + $924.85 + $5000.9 + $1940.9 + $550.33 + $440.44 + $333.33 + $22222.22 is $31768.4.'}

In [16]:
from typing import Any, Type
from langchain.tools import BaseTool
from pydantic import BaseModel, Field

class CalculatorToolArgsSchema(BaseModel): 
    a: int = Field(description="The first number")
    b: int = Field(description="The second number") 
    
class CalculatorTool(BaseTool): 
    name = "CalculatorTool"
    description = """
        Use this to perform sums of two numbers. 
        The first and second arguments should be numbers.  
        Only receives two arguments
    """
    args_schema : Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema

    def _run(self, a, b):
        return plus(a, b) 
    

prompt = "Cost of $35543 + $92485 + $50009 + $1940.9 + $55033 + $44044 + $33333 + $2222222"
    
agent = initialize_agent(llm=llm, verbose=True, agent=AgentType.OPENAI_FUNCTIONS, tools=[CalculatorTool()])

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `CalculatorTool` with `{'a': 35543, 'b': 92485}`


128028
Invoking: `CalculatorTool` with `{'a': 128028, 'b': 50009}`


178037
Invoking: `CalculatorTool` with `{'a': 178037, 'b': 1940.9}`




ValidationError: 1 validation error for CalculatorToolArgsSchema
b
  Input should be a valid integer, got a number with a fractional part [type=int_from_float, input_value=1940.9, input_type=float]
    For further information visit https://errors.pydantic.dev/2.7/v/int_from_float